# <span style="color:gold">Algoritmi di Crittografia (2023/24)</span> 
## Notebook 7 

In [ ]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('/home/mauro/.jupyter/custom/custom.css').read()))

Latex definitions
$\DeclareMathOperator*{\prob}{prob}$
$\DeclareMathOperator*{\mod}{mod}$
$\DeclareMathOperator*{\ln}{ln}$
$\def\zn{\mathbf{Z}_n}$
$\def\zp{\mathbf{Z}_p}$
$\def\mcd{\mathrm{MCD}}$

In [ ]:
# Import delle funzioni/moduli utilizzati nel notebook
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Util import number

### <span style="color:gold">Firma digitale</span>
* La <span style="color:gold">firma digitale</span> (<span style="color:gold">digital signature</span>) è il metodo di autenticazione basato su crittografia a chiave pubblica. 
* In questo contesto è, almeno in linea di principio, molto semplice da spiegare. Si tratta infatti di una <span style="color:gold">"cifratura" effettuata con la chiave privata</span>, anziché con quella pubblica. 
* L'autenticità del mittente di un messaggio $M$ deriva quindi dal fatto che solo con la corrispondente chiave pubblica $M$ può essere "decifrato".
* In realtà poi, a seconda del sistema usato, i "dettagli" possono non essere proprio banali

### <span style="color:cyan">Firma digitale con RSA</span>

* Iniziamo dall'RSA perché, in questo caso, i dettagli matematici (di base) sono proprio gli stessi visti per la cifratura. 
* L'osservazione fondamentale è il ruolo <span style="color:gold">perfettamente simmetrico degli esponenti $e$ e $d$</span> 
* Essi sono uno l'inverso dell'altro in $\mathbf{Z}_n^*$ e nella dimostrazione di correttezza del processo di decifrazione entrano in gioco solo nel prodotto $e\cdot d$. 
* Ne consegue che, prescindendo da altre considerazioni, nel caso dell'RSA le virgolette di cui sopra potrebbero proprio essere eliminate: la firma digitale viene infatti apposta precisamente <span style="color:gold">cifrando il messaggio con la chiave segreta</span> anziché con quella pubblica, cioè utilizzando l'identico algoritmo con $d$ al posto di $e$.

$$
F = M^d\ \mathrm{mod}\ n
$$

* Naturalmente non si tratta di una cifratura (e infatti non abbiamo usato il simbolo $C$, bensì $F$, iniziale di <span style="color:gold">firma</span>) perché il messaggio così trasformato <span style="color:gold">non è reso confidenziale</span>, poiché può essere rimesso in chiaro mediante l'esponente $e$ che è pubblicamente noto. 
* D'altra parte, la confidenzialità non è qui l'obiettivo da raggiungere. 
* Questo procedimento da solo <span style="color:gold">non basta poi per essere certi dell'autenticità di $F$</span>. 
* Che cosa vuol dire qui, infatti, "rimettere in chiaro" $F$? visto che, matematicamente, è sempre possibile applicare la trasformazione:
$$
F^e\ \mathrm{mod}\ n
$$
e dunque <span style="color:gold">qualcosa si ottiene sempre</span>! 

* Per essere sicuri della provenienza dobbiamo quindi avere anche una <span style="color:gold">copia del messaggio originale</span> in modo da poter effettuare un confronto. 
* La firma digitale di un messaggio $M$ è dunque una <span style="color:gold">coppia $(M,F)$</span>, dove $F=M^d\ \mathrm{mod}\ n$
* La garanzia che la coppia sia da ricondurre al proprietario di $e$ deriva dal fatto che, senza disporre di $d$, si ritiene computazionalmente intrattabile <span style="color:gold">forgiare una coppia $(\bar{M},\bar{F})$</span>, dove $\bar{F}=\bar{M}^d\ \mathrm{mod}\ n$.

* Possiamo dunque enucleare la basi del protocollo di autenticazione con firma digitale RSA.

### <span style="color:cyan">Protocollo base</span>

<br />

#### <span style="font-style: italic; color:cyan">Firma del messaggio (Alice)</span>

1. Calcola $F=M^d\ \mathrm{mod}\ n$
2. invia la coppia $(M,F)$ a Bob

<br />

#### <span style="font-style: italic; color:cyan">Verifica della firma (Bob)</span>
1. Si procura la chiave pubblica $(e,n)$ di Alice
2. calcola $M'=F^e\ \mathrm{mod}\ n$
3. accetta se e solo se $M'=M$

* In un'implementazione "reale" di qualsiasi protocollo di firma digitale, la coppia che viene inviata da Alice a Bob non include la firma diretta del messaggio $M$ bensi di un suo <span style="color:gold">fingerprint</span> ottenuto mediante l'applicazione di una qualche funzione hash crittografica (es. SHA256)
* Questo ha il vantaggio di <span style="color:gold">ridurre sensibilmente la dimensione dei numeri</span> ai quali viene applicato il calcolo dell'esponenziale e consente anche la firma di messaggi arbitrariamente lunghi.
* Tutto questo <span style="color:gold">senza perdita di sicurezza</span> a patto che la funzione hash sia resistente alle <span style="color:gold">collisioni</span> 
* Deve cioè essere <span style="color:gold">computazionalmente proibitivo</span> trovare, dato $M$, un altro messaggio $\bar{M}$ tale che $\mathrm{hash}(\bar{M})=\mathrm{hash}(M)$.
* Se questo è il caso si dice che la funzione hash è <span style="color:gold">second pre-image resistant</span>

#### <span style="color:cyan">Protocollo base con hash</span>

<br />

#### <span style="font-style: italic; color:cyan">Firma del messaggio (Alice)</span>

1. Calcola $H=\mathrm{hash}(M)$
2. Calcola $F=H^d\ \mathrm{mod}\ n$
3. invia la coppia $(M,F)$ a Bob

<br />

#### <span style="font-style: italic; color:cyan">Verifica della firma (Bob)</span>
1. Si procura la chiave pubblica $(e,n)$ di Alice
2. calcola $H'=F^e\ \mathrm{mod}\ n$
3. calcola $H=\mathrm{hash}(M)$
3. accetta se e solo se $H'=H$

### <span style="color:cyan">Blinding attack al protocollo base</span>

* Un attacco potenzialmente serio al protocollo base è quello chiamato <span style="font-syle: italic; color:gld">blinding attack</span>.
* L'obiettivo è di riuscire ad ottenere la firma del soggetto attaccato (supponiamo sia Alice) su un messaggio $M$ che ella sicuramente <span style="color:gold">non firmerebbe mai</span>. 
* Per avere successo l'attacco richiede almeno due condizioni (una ottenibile solo con <span style="color:gold">ingegneria sociale</span>). 
* Una è l'equivalente della possibilità di scegliere il testo cifrato in un attacco alla confidenzialità. 
* Dobbiamo supporre di poter ottenere che Alice firmi un messaggio $\bar{M}$ il cui contenuto ella condivide (poniamo, una petizione, una richiesta da parte di un gruppo a un qualche organismo pubblico, o simili) ma che (questa è la <span style="color:gold">"condizione tecnica"</span>) nasconda in qualche modo il messaggio $M$ pericoloso per Alice. 
* Vediamo con ordine.

<br />

#### <span style="color:cyan">Blinding attack (Eva)</span>

<ol>
    <li>Cerca un numero $R$ tale che $\bar{M} = R^e\cdot M\ \mathrm{mod}\ n$ è un messaggio che, quando firmato da Alice, comporta dei vantaggi per quest'ultima;</li>
    <li>"Convince" Alice a firmarlo, ottenendo $F=\bar{M}^{d}\ \mathrm{mod}\ n$;</li>
    <li>Poiché $F=\bar{M}^{d}\ \mathrm{mod}\ n = \left(R^e\cdot M\ \mathrm{mod}\ n\right)^{d}\ \mathrm{mod}\ n=R^{ed}\cdot M^d\ \mathrm{mod}\ n = R\cdot M^d\ \mathrm{mod}\ n$, a Eva è sufficente calcolare $F\cdot R^{-1}\ \mathrm{mod}\ n$ per ottenere proprio $M^d\ \mathrm{mod}\ n$</li>
</ol>

* La parte più difficile è proprio il primo punto, ovvero nascondere il messaggio pericoloso "dentro" un messaggio apparentemente innocuo.
* Ma è tutto l'attacco a sfumare quando si utilizzi il <span style="color:gold">valore hash </span>del messaggio per eseguire la firma. 
* In effetti non sono noti attacchi vincenti al protocollo base con hash.
* Per "buona misura", comunque, le possibilità di un attacco di avere successi si riducono ulteriormente con l'utilizzo di <span style="color:gold">"padding" casuale</span>.

#### <span style="color:cyan">Lo standard PSS (Probabilistic Signature Scheme).</span>
 
* L'idea in se è semplice. Se si applica una funzione hash al messaggio, in generale (di fatto sempre) si ottiene una sequenza di bit che è di <span style="color:gold">molto inferiore</span> alla lunghezza massima dei messaggi consentiti dalla lunghezza degli attuali moduli RSA. 
* Lo "spazio" rimanente può essere riempito (padded) con bit casuali e il calcolo RSA vero e proprio applicato all'hash del messaggio opportunamente "padded".
* I dettagli sono un pò più "tediosi" (stile RSA-OAEP) e li omettiamo.

#### <span style="color:magenta">Esempio di uso di RSA-PSS con il package Python Crypto</span>

In [ ]:
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes

#### <span style="font-style: italic; color:magenta">Generazione ed esportazione delle chiavi</span>

In [ ]:
key = RSA.generate(2048)
# Esportazione della chiave pubblica su file
with open('BobKey.pem','wb') as f:
    f.write(key.publickey().exportKey())
# Esportazione della chiave privata su file con password di protezione
privateKey = \
key.exportKey(passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')
with open('BobSecretKey.pem','wb') as f:
    f.write(privateKey)

#### <span style="font-style: italic; color:magenta">Preparazione e firma del messaggio con RSA-PSS</span>

In [ ]:
message = b'Try this, if you can!'
key = RSA.import_key(open('BobSecretKey.pem').read(),passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')
h = SHA256.new(message)
keylen = key.size_in_bytes()           # Lunghezza (in byte) della chiave    
digestlen = h.digest_size              # Lunghezza dell'output della funzione hash
saltlen = keylen - digestlen -2        # Lunghezza massima del salt. La minima è 0 -> schema deterministico
signer = pss.new(key, salt_bytes = saltlen, rand_func = get_random_bytes)
signature = signer.sign(h)
# Invia la coppia (m,signature)

#### <span style="font-style: italic; color:magenta">Verifica dell'autenticità del messaggio firmato</span>

In [ ]:
# riceve la coppia (m,signature)
key = RSA.import_key(open('BobKey.pem').read())   # import della chiave
h = SHA256.new(message)                           # digest del messaggio
keylen = key.size_in_bytes()                      # 
digestlen = h.digest_size                         # calcolo della lunghezza del salt (per il padding)
saltlen = keylen - digestlen -2                   #
verifier = pss.new(key, salt_bytes = saltlen, rand_func = get_random_bytes)
try:
    verifier.verify(h, signature)
    print("La firma è autentica.")
except (ValueError, TypeError):
    print("La firma non è autentica.")

### <span style="color:cyan">Firma digitale con il protocollo di ElGamal</span>

* Come per i messaggi cifrati, anche quelli firmati con il protocollo di Elgamal sono protetti dalla presumibile intrattabilità del problema del calcolo del <span style="color:gold">logaritmo discreto</span> in opportuni gruppi ciclici. 
* Noi abbiamo studiato il protocollo di cifratura (e ora vedremo quello per la firma digitale) soprattutto per <span style="color:gold">ragioni storico/didattiche</span>
* Nella documentazione del package Python <span style="color:gold">Crypto</span> possiamo infatti leggere quanto segue.

<div style="text-align: center;">
    <div style="display: inline-block; text-align: left;">
        Even though ElGamal algorithms are in theory reasonably secure, 
        <br />
        in practice there are no real good reasons to prefer them to RSA instead
    </div>
</div>

* Ci limitiamo quindi all'aspetto matematico. 
* Diversamente dall'RSA, il protocollo per la firma <span style="color:gold">non è identico</span> a quello di cifratura. 
* Le chiavi pubblica e privata (che, al solito, supporremo di Alice) sono ovviamente le stesse ma le ricordiamo per comodità di riferimento.

<ol>
    <li><span style="color:gold">Chiave segreta</span>. Oltre ai valori $p$ e $g$ resi disponibili come parte della chiave pubblica, un numero $a\in \{2,\ldots,p-2\}$, scelto uniformemente a caso da Alice.</li>
    <li><span style="color:gold">Chiave pubblica</span>. &Egrave; composta da tre numeri (1) il modulo $p$ che definisce il gruppo $\mathbf{Z}_p^*$; (2) una radice primitiva $g$ del gruppo; (3) il valore $A=g^a\ \mathrm{mod}\ p$, dove $a$ è la chiave segreta.</li>
</ol>

#### <span style="color:cyan">Firma del messaggio (Alice)</span>

1. Sceglie un numero $k$ a caso ma tale che <span style="color:gold">$\mathrm{MCD}(k,p-1)=1$</span>
2. calcola le due quantità <span style="color:gold">$r=g^k \ \mathrm{mod}\ p$</span> e <span style="color:gold">$s=k^{-1}\left(M-a\cdot r\right)\ \mathrm{mod}\ (p-1)$</span>
3. invia a Bob il <span style="color:gold">messaggio firmato $(M,(r,s))$</span>

<br />

#### <span style="color:cyan">Verifica della firma (Bob)</span>
1. Si procura la <span style="color:gold">chiave pubblica $(p,g,A)$</span> di Alice
2. calcola le due quantità <span style="color:gold">$x_1 = A^r\cdot r^s\ \mathrm{mod}\ p$</span> e <span style="color:gold">$x_2 = g^M\ \mathrm{mod}\ p$</span>
3. accetta la firma come autentica se e solo se <span style="color:gold">$x_1=x_2$</span>

#### <span style="color:cyan">Correttezza</span>
* Vediamo dapprima a che cosa corrisponde la quantità <span style="color:gold">$r^s\ \mathrm{mod}\ p$</span>

\begin{eqnarray*}
r^s\ \mathrm{mod}\ p&=&\left(g^k\ \mathrm{mod}\ p\right)^s\ \mathrm{mod}\ p\\
&=&\left(g^k\right)^{k^{-1}\left(M-a\cdot r\right)\ \mathrm{mod}\ (p-1)}\ \mathrm{mod}\ p\\
&=&\left(g^k\right)^{k^{-1}\left(M-a\cdot r\right)-(p-1)h}\ \mathrm{mod}\ p\\
&=&\left(g^k\right)^{k^{-1}\left(M-a\cdot r\right)}\left(g^k\right)^{-(p-1)h}\ \mathrm{mod}\ p\\
&=&g^{M-a\cdot r}\left(g^{-kh}\right)^{p-1}\ \mathrm{mod}\ p\\
&=&g^{M-a\cdot r}\left(\left(g^{-kh}\right)^{p-1}\ \mathrm{mod}\ p\right)\ \mathrm{mod}\ p\\
&=&g^{M}g^{-a\cdot r}\ \mathrm{mod}\ p\\
&=&g^{M}\left(g^{-a}\ \mathrm{mod}\ p\right)^r\ \mathrm{mod}\ p\\
&=&g^{M}A^{-r}\ \mathrm{mod}\ p\\
\end{eqnarray*}
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dove $h$ (nel terzo passaggio) è il <span style="color:gold">quoziente</span> della divisione di $k^{-1}\left(M-a\cdot r\right)$ per $p-1$.
* A questo punto è evidente che, in mancanza di tentativi "forgiatura" (o errori non malevoli):
$$x_1=A^r\cdot r^s\ \mathrm{mod}\ p=A^r\cdot (r^s\ \mathrm{mod}\ p)\ \mathrm{mod}\ p=A^r\cdot \left(g^{M}A^{-r}\ \mathrm{mod}\ p\right)\ \mathrm{mod}\ p=g^{M}\ \mathrm{mod}\ p$$

#### <span style="color:cyan">Sicurezza</span>

* Qui consideriamo solo alcuni fra i più importanti accorgimenti <span style="color:gold">necessari</span> per la sicurezza.

#### <span style="color:cyan">Uso singolo di $k$</span>

* Il valore segreto $k$ deve essere utilizzato una sola volta. 
* Si noti, preliminarmente che, <span style="color:gold">se la firma è autentica</span>, $x_1=x_2$ e ovviamente vale anche <span style="color:gold">$\log_g x_1\mod p=\log_g x_2\mod p=M\mod p$</span>. Da questo, con semplici passaggi
\begin{eqnarray*}
\log_g x_1\mod p&=&\log_g A^r\cdot r^s\ \mathrm{mod}\ p\\
&=&(r\cdot \log_g A+s\cdot\log_g r)\mod p\\
&=&(r\cdot a+s\cdot k)\mod p\\
&=&M\mod p
\end{eqnarray*}
* L'attaccante può dunque utilizzare due messaggi, $M_1$ e $M_2$, firmati con lo stesso valore $k$, e impostare il seguente <span style="color:gold">sistema di due equazioni</span> nelle due incognite $a$ e $k$:
$$
\left\{\begin{array}[rcl]
\ a\cdot r_1 + k\cdot s_1&=&M_1\quad (\mathrm{mod}\ p)\\
\\
a\cdot r_2 + k\cdot s_2&=&M_2\quad (\mathrm{mod}\ p)\\
\end{array}
\right.
$$
* In questo modo, l'unica soluzione fornisce, oltre a $k$, anche il <span style="color:gold">valore della chiave segreta $a$</span> 
* Se invece $k$ è sempre diverso, qualsiasi sia il numero $\ell$ di equazioni considerate, il numero di incognite è sempre $\ell+1$ e dunque il <span style="color:gold">sistema è indeterminato</span>

#### <span style="color:cyan">Valore hash del messaggio</span>

* Il protocollo deve essere usato con <span style="color:gold">hash del messaggio</span>
* In caso contrario diviene possibile portare un <span style="color:gold">message forgery attack</span>, cioè produrre un messaggio correttamente firmato senza conoscere la chiave privata
* Si possono infatti determinare in anticipo i valori $r$ ed $s$ e <span style="color:gold">costruire il messaggio</span> che ha $r$ ed $s$ come firma
* L'attaccante sceglie inizialmente due numeri <span style="color:gold">$x$ e $y$</span>, con <span style="color:gold">$\mathrm{MCD}(y,p-1)=1$</span>, dopodiché pone
$$
r = g^x\cdot A^y\ \mathrm{mod}\ p =g^x\cdot\left(g^a\ \mathrm{mod}\ p\right)^y\ \mathrm{mod}\ p=g^{x+a\cdot y}\ \mathrm{mod}\ p
$$
e
$$
s=-r\cdot y^{-1} \ \mathrm{mod}\ (p-1)
$$
da cui può derivare il valore del messaggio <span style="color:gold">$M$ di cui $(r,s)$ è la firma</span> 
* Eva sa infatti che Bob eseguirà il seguente controllo:
$$
A^r\cdot r^s\ \mathrm{mod}\ p \stackrel{\mathrm{?}}{=} g^M \ \mathrm{mod}\ p
$$
e dunque pone <span style="color:gold">$M=x\cdot s$ $(\mathrm{mod}\ (p-1))$</span>. 
* In questo modo risulta infatti:
\begin{eqnarray*}
A^r\cdot r^s\ \mathrm{mod}\ p&=&\left(g^a\ \mathrm{mod}\ p\right)^r\cdot \left(g^{x+a\cdot y}\ \mathrm{mod}\ p\right)^{s}\ \mathrm{mod}\ p\\
&=&g^{a\cdot r}\cdot g^{(x+a\cdot y)\cdot s}\ \mathrm{mod}\ p\\
&=&g^{a\cdot r + x\cdot s + a\cdot y\cdot s}\ \mathrm{mod}\ p\\
&=&g^{x\cdot s}\mod p\\
&=&g^{M+h(p-1)}\mod p\\
&=&g^{M}\mod p
\end{eqnarray*}
* La correttezza del terz'ultimo passaggio è una conseguenza della <span style="color:gold">scelta di $s$</span>
* Infatti, poiché $s=-r\cdot y^{-1} \ \mathrm{mod}\ (p-1)$ risulta <span style="color:gold">$y\cdot s= -r+h'\cdot(p-1)$</span>, con $h'$ opportuno valore intero $h'$, e dunque
$$
g^{a\cdot r+ a\cdot y\cdot s}\ \mathrm{mod}\ p = g^{a\cdot r-a\cdot r+a\cdot h'\cdot(p-1)}\mod p=1
$$


* Per "parare" questo attacco si usa firmare non il messaggio ma il valore hash di una quantità che <span style="color:gold">include il messaggio</span>
* Più precisamente, dopo avere scelto $k$ e calcolato $r$, <span style="color:gold">Alice firma $H(M||r)$</span>, dove || indica la concatenazione 
* Con questa scelta, Eva non può ovviamente porre $M=x\cdot s\mod (p-1)$ come in precedenza, perché non è questo che Bob usa nella verifica.
* Eva dovrebbe trovare un messaggio $M$ tale che <span style="color:gold">$H(M||r)=x\cdot s\mod (p-1)$</span>, cosa che richiede che <span style="color:gold">$H$ non sia first pre-image resistant</span>

#### <span style="color:magenta">Controllo sul valore di $r$</span>

* &Egrave; necessario che il protocollo di verifica della firma preveda il <span style="color:gold">controllo che $r\in\{1,2,\ldots,p-1\}$</span>
* In caso contrario, diviene agevole per Eva produrre <span style="color:gold">firme apparentemente autentiche su qualsiasi messaggio</span>, nel momento in cui disponga della firma autentica $(r,s)$ di un solo messaggio $M$

* Infatti, <span style="color:gold">scelto il messaggio $M'$</span> da far passare come autentico, Eva calcola preliminarmente il valore
$$
x = M'\cdot M^{-1}\ \mathrm{mod}\ (p-1)
$$
dopodiché sceglie i due valori $r'$ ed $s'$, che costituiranno la firma di $M'$, nel modo indicato di seguito
* Nel caso di $s'$ la scelta è semplice
$$
s' \equiv s\cdot x \ \mathrm{mod}\ (p-1)
$$
* $r'$ deve invece essere scelto in modo che <span style="color:gold">soddisfi simultaneamente le due seguenti equazioni</span>:
$$
r'\equiv r\cdot x\ \mathrm{mod}\ (p-1)\qquad\mathrm{e}\qquad r'\ \mathrm{mod}\ p=r
$$
* Una soluzione con queste proprietà si può ottenere semplicemente ponendo <span style="color:gold">$\alpha = r\cdot(x-1)$</span> e poi <span style="color:gold">$r'=r+\alpha p$</span>
* La verifica che <span style="color:gold">$r'\ \mathrm{mod}\ p=r$</span> è immediata
* Per la prima proprietà abbiamo
\begin{eqnarray*}
r'-rx&=&r+\alpha p-rx\\
&=&r+r(x-1)p-rx\\
&=&-r(x-1)+r(x-1)p\\
&=&r(x-1)(p-1)
\end{eqnarray*}
e dunque <span style="color:gold">$r'-rx$ è un multiplo di $p-1$</span>, come richiesto
* La dimostrazione che $M'$, con la firma $(r',s')$ passa la verifica è lasciata come <span style="color:gold">semplice esercizio di "manovra" dei moduli</span>

* Per parare l'attacco è necessario controllare che la componente $r$ della firma sia un numero <span style="color:gold">minore del modulo $p$</span>
* In una firma legale, infatti, $r$ è definito come <span style="color:gold">$g^k\mod p$</span>
* Al contrario, $r'=r+\alpha p$ è <span style="color:gold">quasi sempre un numero maggiore di $p$</span> e dunque il controllo di cui sopra è quasi sempre efficace
* Ci sono tuttavia un paio di casi speciali che vanno controllati
* Teniamo tuttavia presente che dal lato dell'attaccante è necessario che <span style="color:gold">$M$ abbia inverso modulo $p-1$</span> e dunque, in particolare, che sia strettamente minore di $p-1$

* Veniamo ai casi speciali:
    1. Se $x=1$ risulta $\alpha=0$ e $r'=r$; tuttavia $x=1$ richiede <span style="color:gold">$M'=M\mod\ (p-1)$</span> e dunque, poiché $M<p-1$, risulta propriamente <span style="color:gold">$M'=M$ e l'attacco svanisce</span>
    2. Se $x=0$ risulta $\alpha=-r$ e $r'=r-rp$. $x=0$ implica che il messaggio "falso" sia un multiplo di $p-1$ e questo <span style="color:gold">potrebbe effettivamente costituire una minaccia</span>. Ora, è vero che un valore negativo di $r$ fa <span style="color:gold">fallire</span> l'esponenziale modulare (presente nel calcolo di <span style="color:gold">$A^rr^s\mod p$</span>); tuttavia Eva, che conosce molto bene l'aritmetica modulare, in questo caso non presenta $r'$ bensi <span style="color:gold">$r'\mod\ p-1=0$</span>. Questa modifica non può essere fatta sempre perché cambia anche l'altro fattore presente nel calcolo di $A^rr^s\mod p$, e cioè <span style="color:gold">$r^s\mod p$</span>. L'accorgimento di Eva funziona solo in questo caso, in quanto anche <span style="color:gold">$s=0$</span>.

In [ ]:
from AClibrary.crypto import EGKey
from Crypto.Math.Numbers import Integer

In [ ]:
key = EGKey(256)
publickey = key.publickey()

In [ ]:
text = b'un messaggio'

In [ ]:
M,(r,s) = key.sign(text)

In [ ]:
M,(r,s)

In [ ]:
publickey.verify(M,r,s)

In [ ]:
ord('n')

In [ ]:
publickey.verify(Integer.from_bytes(b'uo messaggio'),r,s)

In [ ]:
def r_attack(key,Mprime,M,r,s):
    '''Attacco basato sulla mancata verifica del valore di r'''
    if Integer.gcd(M,key.p-1)>1:
        print("Impossibile portare l'attacco")
        return
    M1=Integer.inverse(M,key.p-1)  # inversione di M modulo p-1
    x = (Mprime*M1)%(key.p-1)      # calcolo del valore x
    if x==0:
        sp = Integer(0)
        rp = (r-r*key.p)%(key.p-1)
    else:
        sp=(s*x)%(key.p-1)
        alpha=r*(x-1)
        rp = r+alpha*key.p
    return rp,sp

In [ ]:
fakemsg = b'I owe Eve 10000 bucks'
Mprime = Integer.from_bytes(fakemsg)

In [ ]:
rp,sp = r_attack(publickey,Mprime,M,r,s)

In [ ]:
publickey.verify(Mprime,rp,sp)

* Osserviamo che cosa accade se $M'\mod\ p-1=0$

In [ ]:
Mprime = Integer(34)*(publickey.p-1)

In [ ]:
rp,sp = r_attack(publickey,Mprime,M,r,s)

In [ ]:
rp,sp

* In questo caso, come previsto

In [ ]:
publickey.verify(Mprime,rp,sp)

### <span style="color:gold">Firma digitale con DSA (Digital Signature Algorithm)</span>
* L'algoritmo DSA è un metodo definito con precisione da <span style="color:gold">NIST</span> (National Institute of Standards and Technology) 
* &Egrave; stato adottato come <span style="color:gold">standard</span> nel 1994 (FIPS PUB 186)
* Versione attuale rilasciata nel luglio 2013 (FIPS PUB 186-4)
* Come al solito vediamo le tre componenti di <span style="color:gold">generazione chiavi, firma e verifica</span>
* A differenza di altri protocolli (di genesi accademica e dunque inizialmente presentati nella loro essenza matematico/algoritmica) DSA prevedeva "da subito" l'<span style="color:gold">uso di una funzione hash</span> sul messaggio da firmare
* Nello specifico, la funzione hash era $\mathtt{SHA-1}$

### <span style="color:cyan">Protocollo originale</span>

<br />

#### <span style="color:cyan">Generazione delle chiavi (Alice)</span>

1. Genera un numero <span style="color:gold">primo $q$ di 160 bit</span>
2. Genera un numero <span style="color:gold">primo $p$ di 1024 bit</span> tale che $q$ sia un divisore primo di $p-1$
3. Determina un elemento $g\in\zp^*$ di ordine $q$ (in altri termini, un <span style="color:gold">generatore del sottogruppo di $\zp^*$ di $q\approx 2^{160}$ elementi</span>)
4. Sceglie a caso un numero <span style="color:gold">$a\in\mathbf{Z}_q$</span>
5. Calcola <span style="color:gold">$A=g^a\mod\ p$</span>
6. Pubblica <span style="color:gold">$(p,q,g,A)$</span> e tiene riservato il numero $a$

<br />

#### <span style="color:cyan">Firma del messaggio (Alice)</span>

1. Dato il messaggio $M$, calcola <span style="color:gold">$m=\mathtt{SHA-1}(M)$</span> (160 bit)
2. Sceglie <span style="color:gold">$k$ uniformemente a caso in $\mathbf{Z}_q^*$</span>
3. Calcola <span style="color:gold">$r=(g^k\mod p)\mod q$</span> e <span style="color:gold">$s=k^{-1}(m+a\cdot r)\mod q$</span>
4. Se anche uno solo dei due valori ($r$ o $s$) è 0, sceglie un <span style="color:gold">diverso valore di $k$</span> (passo 2) 
5. Altrimenti invia il messaggio $M$ insieme alla <span style="color:gold">firma $(r,s)$</span>

<br />

#### <span style="color:cyan">Verifica della firma (Bob)</span>
1. Si procura la chiave pubblica $(p,q,g,A)$ di Alice
2. Controlla che risulti $0<r,s<q$
3. Calcola <span style="color:gold">$x = \mathtt{SHA-1}(M)\cdot(s^{-1}\mod q)$</span>
4. Calcola <span style="color:gold">$y = r\cdot(s^{-1}\mod q)$</span>
5. Esegue il controllo <span style="color:gold">$(g^xA^y\mod p)\mod q\stackrel{\mathrm{?}}{=}r$</span> e accetta la firma come autentica in caso di uguaglianza

#### <span style="color:cyan">Un esempio illustrativo con numeri piccoli</span>

In [2]:
from ACLIB.utils import isPrime, modular_inverse
from random import randint

In [6]:
#######################################################
##### Generazione delle chiavi privata e pubblica #####
#######################################################
while (q := int(input("Inserisci il valore di q: "))) and not isPrime(q):
    pass
# Determinazione di un primo p t.c. q divide p-1
r = 2*q+1  # Arbitrario valore di partenza
while (p:=2*q*r+1) and not (isPrime(r) and isPrime(p)):
    r += 2
# Determinazione di un generatore del sottogruppo di q elementi
h = 2
r2 = (p-1)//q
while (g:=h**r2 % p) and g == 1:
    h+=1
# Scelta (casuale) della chiave privata
a = randint(1,q-1)
# Calcolo dell'ultimo valore della chiave privata
A = g**a % p
print('Public key')
print(f'q={q}\tp={p}\tg={g}\tA={A}')

Inserisci il valore di q: 17
Public key
q=17	p=1259	g=316	A=83


In [7]:
g**q%p

1

In [8]:
h

2

In [18]:
#######################################################
## Firma di un messaggio (visto come già "hash-ato") ##
#######################################################
while (m:=int(input(f'Inserisci il valore del messaggio (m<{q}): '))) and (m>=q or m<0):
    pass
while True:
    k = randint(1,q-1)
    r = (g**k % p) % q
    s = (modular_inverse(k,q)*(m+r*a))%q
    #if r%q != 0 and s%q != 0:
    if r%q == 0 or s%q == 0:
        break
print('Digital signature')
print(f'msg={m}\tr={r}\ts={s}')

Inserisci il valore del messaggio (m<17): 3
Digital signature
msg=3	r=5	s=0


In [11]:
#######################################################
####### Verifica dell'autenticità del messaggio #######
#######################################################
m_rcvd,r_rcvd,s_rcvd = 16,r,s
x = m_rcvd*modular_inverse(s_rcvd,q)
y = r_rcvd*modular_inverse(s_rcvd,q)
rprime = ((g**x)*(A**y)% p) % q
if r_rcvd == rprime:
    print("Accept")
else:
    print("Reject")

Reject


#### <span style="color:cyan">Due semplici esercizi</span>
1. Dimostrare che, se $k$ (che pure è ephemeral) è stata compromessa, anche la chiave privata di Alice è compromessa
2. Dimostrare che, inviando un messaggio firmato con $r$ o $s$ uguali a 0, la chiave privata viene compromessa

In [ ]:
# Attacco quando k è noto e entrambi i valori r ed s sono non nulli
aprime = ((k*s-m)*modular_inverse(r,q))%q
aprime == a

In [13]:
# Attacco quando r = 0 ma s è non nullo
kprime = (modular_inverse(s,q)*m)%q
kprime == k

True

In [19]:
# Attacco quando r è non nullo e s=0
aprime = (q-m)*modular_inverse(r,q)%q
aprime==a

True

* <span style="color:gold">Osservazione:</span> il caso $(s\equiv 0)\mod q$ implica comunque che la verifica non è possibile nel modo "standard". Se $r$ è non nullo, la verifica può essere fatta da Alice recuperando prima $a$ (nel modo appena visto) e poi verificando che $r$ soddisfa l'equazione
$$
m+r\cdot a=0\quad\mod q
$$

In [ ]:
(m+r*a)%q

* Se poi anche $r$ è nullo, allora questo implica che $m=0 \mod q$.

#### <span style="color:cyan">Sicurezza</span>
* La sicurezza del protocollo DSA dipende dalla difficoltà di calcolo del <span style="color:gold">logaritmo discreto</span> su un sottogruppo di $q$ elementi
* Il protocollo orginale, in qui $q$ viene scelto nell'intervallo $(2^{159},2^{160})$ fornisce quindi <span style="color:gold">$\log(\sqrt{2^{160}})=80$ bit di sicurezza</span>, sulla base delle attuali conoscenze riguardo la difficoltà di calcolo del logaritmo discreto

#### <span style="color:cyan">Specifiche del protocollo attuale</span>
* La lunghezza del modulo $p$ può essere uno dei seguenti valori: <span style="color:gold">1024, 2048, o 3072 bit</span>
* La lunghezza in bit di $q$ è uno dei valori <span style="color:gold">160, 224 o 256</span> 
* Come funzione hash può essere utilizzata qualsiasi funzione specificata nelle pubblicazioni <span style="color:gold">FIPS 180</span>

#### <span style="color:cyan">Un esempio con il package Crypto</span>

In [ ]:
from Crypto.PublicKey import DSA
from Crypto.Hash import SHA256,SHA224
from Crypto.Signature import DSS
from math import log2,ceil

In [ ]:
key = DSA.generate(2048)
ceil(log2(key.q))

In [ ]:
key.__dict__  # Il valore di x è quello che nella descrizione del protocollo abbiamo chiamato a

In [ ]:
message = b"Hello"
m = SHA224.new(message)

In [ ]:
signer = DSS.new(key,mode='fips-186-3') # Il modo prevede la scelta randomizzata di k con RNG
signature = signer.sign(m)

* Alice invia la coppia message,signature
* Bob si procura la chiave pubblica di Alice

In [ ]:
pubkey = key.publickey()

In [ ]:
pubkey.__dict__

* A questo punto può ricalcolare l'hash del messaggio ricevuto ed effettuare la verifica

In [ ]:
m = SHA224.new(message)
#m =  SHA256.new(b'Helln')

In [ ]:
verifier = DSS.new(pubkey, 'fips-186-3')

In [ ]:
try:
    verifier.verify(m, signature)
    print("The message is authentic.")
except ValueError:
    print("The message is not authentic.")

### <span style="color:gold">Autenticità delle chiavi pubbliche</span>
* Quando Bob <span style="color:gold">"si procura"</span> (abbiamo usato più volte questa locuzione ...) la chiave pubblica di Alice, vuoi per cifrare un messaggio, vuoi per verificare l'autenticità di un documento, egli deve essere certo che tale chiave pubblica <span style="color:gold">sia effettivamente la chiave di Alice</span>
* Ci sono almeno due approcci per ottenere questo risultato, esemplificati dall'uso che ne fanno le suite di protocolli <span style="color:gold">TLS/OpenSSL</span> e <span style="color:gold">GnuPG/OpenPGP</span>
* Il primo approccio necessita di un'infrastruttura, costituita dalle cosiddette <span style="color:gold">autorità di certificazione</span> (CA)

### <span style="color:cyan">Approccio TLS</span>
* Tale soluzione è bene illustrata nel caso di interazione fra <span style="color:gold">browser (client) e web server</span> 
* L'interazione tipicamente parte per iniziativa del client che contatta il server, ad esempio di un negozio di vendita on-line
* Questo primo contatto è in chiaro e il rischio che in questa fase si verifichino i <span style="color:gold">preliminari di un attacco</span> (ad esempio mediante site-spoofing) è alquanto elevato
* Il server deve rispondere con un messaggio che include un <span style="color:gold">certificato di autenticità</span> 
* Si tratta essenzialmente della <span style="color:gold">chiave pubblica del server firmata</span> da una qualche autorità riconosciuta in grado di fungere da garante per il server
* L'interazione prosegue poi in modi differenti a seconda che il client già possieda, o meno, un certificato del server precedentemente <span style="color:gold">verificato e in corso di validita</span>

1. Se possiede tale certificato, il client possiede <span style="color:gold">anche la chiave pubblica verificata</span> e l'interazione può procedere secondo il protocollo concordato
2. Se non possiede tale certificato ma possiede la chiave pubblica verificata dell'<span style="color:gold">autorità che garantisce per il server</span>, il client acquisisce come autentico il certificato memorizzandolo nel cosiddetto <span style="color:gold">keyring</span> per usi futuri. Anche in questo caso, l'interazione può procedere secondo il protocollo concordato. 
3. Se non possiede neppure la chiave pubblica dell'autorità, il client deve prima autenticare quest'ultima mediante un'autorità di livello superiore che garantisce per essa. 
    * Il problema assume cioè <span style="color:gold">connotati ricorsivi</span>. 
    * Al riguardo va però notato che il certificato presentato dal server deve includere specifica della <span style="color:gold">"catena" di certificazioni</span>, che dal server arriva ad un'autorità top-level che garantisce per le subordinate. 
    * E dunque fondamentale che le chiavi pubbliche delle autorità di massimo livello siano memorizzate nel keyring quando il <span style="color:gold">sistema operativo viene installato</span> (da fonte attendibile)

In [ ]:
!openssl s_client -connect www.google.it:443

### <span style="color:cyan">GnuPG (II)</span>

* All'approccio che caratterizza TLS (brevemente schematizzato sopra) si contrappone quello utilizzato da GnuPG/OpenPGP, la cui organizzazione è essenzialmente <span style="color:gold">orizzontale</span> (potremmo dire di tipo peer-to-peer)
* Si tratta di una <span style="color:gold">soluzione molto più leggera</span> che non potrebbe essere utilizzata per gli scopi in cui si applica TLS (commercio elettronico, home-banking, ...)
* In ambito GnuPG, l'associazione fra persone (e i loro indirizzi email) e chiavi pubbliche avviene tramite il cosiddetto <span style="color:gold">web of trust</span>, ovvero una rete di fiducia 
* Vediamo un esempio concreto. 
    1. Un docente (a caso...) <span style="color:gold">firma i messaggi di posta elettronica</span> destinati, fra gli altri, agli studenti dei suoi corsi, di modo che questi ultimi siano certi della loro provenienza
    2. Se uno studente vuole essere sicuro della fonte può accedere ad un server di chiavi PGP, ad esempio il <span style="color:gold">MIT PGP Public Key Server</span> e recuperare la  chiave pubblica del docente in questione.
    3. Come può però egli/ella essere sicuro che il sito non sia stato "spoof-ato" o che la chiave sia stata modificata nel transito? 
    4. La chiave ha però anche un <span style="color:gold">fingerprint</span>. Per altra via, ad esempio durante una lezione online, il docente comunica tale fingerprint e così gli studenti, ragionevolmente certi di stare interagendo con il docente, possono verificare che la chiave sia autentica. 
    5. A quel punto la <span style="color:gold">importano nel proprio keyring</span> attribuendo ad essa un valore di confidenza elevato. 
    6. Uno studente, non presente durante questa fase pubblica, nota che molti suoi compagni hanno attribuito alta confidenza a quella particolare chiave e decide che questo è <span style="color:gold">sufficiente per "credere" che la chiave appartenga effettivamente a quel docente</span>
    7. A questo punto anch'egli la importa, eventualmente attribuendole un grado di fiducia <span style="color:gold">un poco inferiore a quello dei propri compagni</span>

* Quello delineato sopra è solo un esempio di un possibile scenario in cui il <span style="color:gold">web-of-trust si allarga</span>
* OpenPGP include <span style="color:gold">regole precise per attribuire grado di fiducia</span> più o meno elevato ad una chiave sulla base del grado di fiducia a questa attribuito da altri

#### <span style="color:cyan">Importare una chiave nel keyring</span>
* Per <span style="color:gold">cercare</span> una chiave

In [ ]:
!gpg --keyserver pgp.mit.edu --search leoncini@unimore.it

* Per <span style="color:gold">importare</span> una chiave

```shell
gpg --keyserver pgp.mit.edu --recv-keys 96C0819CD95DCEE8
```

* Per <span style="color:gold">attribuire un grado di fiducia</span> alla chiave importata

```shell
> gpg --edit-key leoncini@unimore.it
gpg (GnuPG) 2.2.27; Copyright (C) 2021 Free Software Foundation, Inc.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Secret key is available.

sec  rsa4096/96C0819CD95DCEE8
     created: 2013-02-01  expires: never       usage: SC  
     trust: ultimate      validity: ultimate
ssb  rsa4096/3626DF78751E14AC
     created: 2013-02-01  expires: never       usage: E   
[ultimate] (1). Mauro Leoncini <leoncini@unimore.it>
[ultimate] (2)  Mauro Leoncini <mauro.leoncini@unimore.it>
[ revoked] (3)  mauro.leoncini (A strong key, 4096 bit long) <leoncini@unimore.it>

gpg> trust
sec  rsa4096/96C0819CD95DCEE8
     created: 2013-02-01  expires: never       usage: SC  
     trust: ultimate      validity: ultimate
ssb  rsa4096/3626DF78751E14AC
     created: 2013-02-01  expires: never       usage: E   
[ultimate] (1). Mauro Leoncini <leoncini@unimore.it>
[ultimate] (2)  Mauro Leoncini <mauro.leoncini@unimore.it>
[ revoked] (3)  mauro.leoncini (A strong key, 4096 bit long) <leoncini@unimore.it>

Please decide how far you trust this user to correctly verify other users' keys
(by looking at passports, checking fingerprints from different sources, etc.)

  1 = I don't know or won't say
  2 = I do NOT trust
  3 = I trust marginally
  4 = I trust fully
  5 = I trust ultimately
  m = back to the main menu

Your decision?
```

* Se abbiamo un'elevata fiducia che una data chiave appartenga alla persona individuata dall'indirizzo email, possiamo rendere noto (e utilizzabile) questo fatto <span style="color:gold">firmando tale chiave</span> (ovviamente con la nostra chiave privata)

```shell
gpg --sign-key D95DCEE8
```

* Dopodiché possiamo inviare la chiave firmata al server (o ai server), il quale <span style="color:gold">fonderà la nostra firma con le altre disponibili per leoncini@unimore.it</span>

```shell
gpg --keyserver pgp.mit.edu --send-keys D95DCEE8
```